## Example of TRUE nanotribological properties workflow

### Background
- Thin films can be used as coatings to reduce coefficient of friction and force of adhesion when shearing between two surfaces.
- Many variables of a thin film can be varied and, hence, makes it a prime candidate for computational simulation.
- Several molecular dynamics (MD) studies on this subject have been conducted, focusing on investigating different properties.
- This specific example is based on a more in-depth study by Dr. Andrew Summers et al., which can be found at:  https://pubs.acs.org/doi/10.1021/acs.jctc.9b01183.
    
### Model
- Initialization of two surfaces coated with thin film - varying mixing composition, backbone chainlength, terminal groups
- Two surface was then energy minimized through a few steps (by LAMMPS and GROMACS)
- The two system is then compressed and sheared against each other (the top surface moving, and the bottom surface fixed)
- Output is analyzed with MDAnalysis

### Simulation Details
- Initialze with MoSDeF
- Fix overlaps with LAMMPS
- Energy Minimize with GROMACS
- NVT Equilibration with GROMACS
- Compress with GROMACS
- Shearing at 5nN with GROMACS
- Data Analysis MDAnalysis and pymbar

#### Notice:
- For the purpose of demonstration, this workflow is designed to work for every machine. Thus we are making the assumption that no `GPU` can be used for the course of simulation, and `CPU` options are defaults for most of our calculation. Hence, the performance for the GROMACS engine is greatly compromised. If the user wished to switch to use `GPU` acceleration, one can delete the `-nb cpu` flags and add `-ntmpi 1` for the cells below.
- It should also be noted that, because of the nature of molecular dynamics simulation, the most of the operation belows are quite computationally expensive. Except for the first three steps, `System initialization`, `Fix overlaps by LAMMPS`, and `Energy minimize by GROMACS`, the other simulations will take longer time to finish on a normal personal computer.
- If ones wish to skip the example simulation and look at the scalability, part of the `Extensible` property of TRUE, they can choose to execute the first two cells, which set up our local package, and continue to the last three cells, which set up the expanded workspace and workflow.
- If running the whole notebook is desirable, we recommend user to change the number of steps for the `NVT Equilibration`, `Compress` and `Shearing at 5nN` so the simulations could finish in a shorter time frame. All the input script files is accessible at `TRUE-nanotribology/workflow/util/mdp_files`. Please note that this could affect the accuracy of the final result.

#### System initialization

In [1]:
# !pip install -e .
import mbuild
from util.helper.functions import system_builder
from util.helper.analysis import calc_nematic_order
import warnings
warnings.filterwarnings("ignore")

_ColormakerRegistry()

/root/miniconda3/envs/simulation36/lib/python3.6/site-packages/signac/common/host.py:19: DeprecatedWarning: SimpleKeyring is deprecated as of 1.3 and will be removed in 2.0. The crypt module is deprecated.
  SESSION_PASSWORD_HASH_CACHE = SimpleKeyring()


In [2]:
# Change into working directory
%cd example_simulation_nitro_amino/

/root/Projects/tribology-simulations/workflow/example_simulation_nitro_amino


In [ ]:
structure = system_builder(seed=1, chainlength=17, terminal_group_1 = 'amino', terminal_group_2 = 'nitro',
                           terminal_group_3 = 'amino', num_chains=100)

In [4]:
structure.visualize()

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

#### Fix overlaps by LAMMPS

Note: The wall time needed to run this step on a 2009 Mac Pro machine with 2 x 2.26 GHz Quad-Core Intel Xeon was 0:28:15

In [ ]:
# Command (lmp_serial) might be different (lmp) depend on the installed lammps version
!lmp_serial -in ../util/mdp_files/in.minimize -log minimize.log

#### Energy minimize by GROMACS

Note: The wall time needed to run this step on a 2009 Mac Pro machine with 2 x 2.26 GHz Quad-Core Intel Xeon was 0:11:58

In [ ]:
# Convert .xtc to .gro
!echo 0| gmx trjconv -s init.gro -f minimize.xtc -o minimize.gro  -b 1.0 -e 1.0
# Grompp
!gmx grompp -f ../util/mdp_files/em.mdp -c minimize.gro -p init.top -n init.ndx -o em.tpr -maxwarn 2
# Run GROMACS
!gmx mdrun -v -deffnm em -s em.tpr -cpi em.cpt -nb cpu

#### NVT Equilibrate by GROMACS

Note: The wall time needed to run this step on a 2009 Mac Pro machine with 2 x 2.26 GHz Quad-Core Intel Xeon was 9:32:38

In [ ]:
# Grompp
!gmx grompp -f ../util/mdp_files/nvt.mdp -c em.gro -p init.top -n init.ndx -o nvt.tpr -maxwarn 2
# Run GROMACS
!gmx mdrun -v -deffnm nvt -s nvt.tpr -cpi nvt.cpt -nb cpu

#### Compress wih GROMACS

Note: The wall time needed to run this step on a 2009 Mac Pro machine with 2 x 2.26 GHz Quad-Core Intel Xeon was 4:43:54

In [ ]:
# Grompp
!gmx grompp -f ../util/mdp_files/compress.mdp -c nvt.gro -p init.top -n init.ndx -o compress.tpr -maxwarn 3
# Run GROMACS
!gmx mdrun -nt 16 -v -deffnm compress -s compress.tpr -cpi compress.cpt -nb cpu

#### Shearing at 5nN

Note: The wall time needed to run this step on a 2009 Mac Pro machine with 2 x 2.26 GHz Quad-Core Intel Xeon was 2days 3:12:10

In [ ]:
# Grompp
!gmx grompp -f ../util/mdp_files/shear_5nN.mdp -c compress.gro -p init.top -n init.ndx -o shear_5nN.tpr -maxwarn 3
# Run GROMACS
!gmx mdrun -v -nt 16 -s shear_5nN.tpr -deffnm shear_5nN -cpi shear_5nN.cpt -cpo shear_5nN.cpt -noappend

#### Calculate nematic order

In [ ]:
# Unwrap stuff
!echo 0 | gmx trjconv -f shear_5nN.part0001.xtc -o shear_5nN-unwrapped.xtc -s shear_5nN.part0001.gro -pbc nojump
# Calculate nematic order
calc_nematic_order(traj_filename="shear_5nN-unwrapped.xtc", top_filename="shear_5nN.part0001.gro", output_filename="shear_5nN-S2.txt", ndx_filename="init.ndx", n_chains=100)

In [ ]:
import matplotlib
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline  

In [ ]:
S2 = np.loadtxt(fname='shear_5nN-S2.txt',skiprows=1)

In [ ]:
plt.plot(S2[:,0],S2[:,1])
plt.xlabel('time (ps)')
plt.ylabel('Bottom Monolayer')

In [ ]:
plt.plot(S2[:,0],S2[:,2])
plt.xlabel('time (ps)')
plt.ylabel('Top Monolayer')

- The two figures above represent the nematic order of our systems throughout the last simulation, shearing at 5 nN. Hence, these plot includes the pre-equlibrium periods, which should be discarded before further calculation.
- We could also determine the equilibrium period automatically by using another library called `pymbar`. This library would also allow user to determine the optimal bin size that could be use for block statistic.

In [ ]:
from pymbar import timeseries
from scipy import stats

In [ ]:
[t01, g1, Neff_max1] = timeseries.detectEquilibration(S2[:,1])
[t02, g2, Neff_max2] = timeseries.detectEquilibration(S2[:,2])
if t01 > t02:
    [t0, g, Neff_max] = [t01, g1, Neff_max1]
else:
    [t0, g, Neff_max] = [t02, g2, Neff_max2]

- From here, we can directly calcualte the average nematic order of the bottom and top monolayer of the production period.

In [ ]:
bottom = [round(np.mean(S2[t0:,1]),3), round(np.std(S2[t0:,1]),3)]
top = [round(np.mean(S2[t0:,2]),3), round(np.std(S2[t0:,2]),3)]
print('Nematic order of:')
print('\tBottom monolayer: {} +/- {}'.format(bottom[0],bottom[1]))
print('\tTop monolayer: {} +/- {}'.format(top[0],top[1]))

- We can also plot the nematic order of this period to make sure there is no odd behavior.

In [ ]:
plt.plot(S2[t0:,0],S2[t0:,1], label='Bottom monolayer')
plt.plot(S2[t0:,0],S2[t0:,2], label='Top monolayer')
plt.xlabel('time (ps)')
plt.ylabel('Nematic order')
plt.ylim(0.4, 0.65)
plt.legend(loc='upper right')

- We can also perform block average to have a more refined plot

In [ ]:
equilibrated_time = S2[t0:, 0]
equilibrated_bottom = S2[t0:, 1]
equilibrated_top = S2[t0:, 2]
indices_top = timeseries.subsampleCorrelatedData(equilibrated_top,conservative=True)
indices_bottom = timeseries.subsampleCorrelatedData(equilibrated_bottom,conservative=True)
if indices_top[2] > indices_bottom[1]:
    indices = indices_bottom
else:
    indices = indices_top

In [ ]:
bottom_block_average = stats.binned_statistic(equilibrated_time,equilibrated_bottom,statistic='mean',bins=equilibrated_time[list(indices)])
bottom_block_std = stats.binned_statistic(equilibrated_time,equilibrated_bottom,statistic='std',bins=equilibrated_time[list(indices)])

top_block_average = stats.binned_statistic(equilibrated_time,equilibrated_top,statistic='mean',bins=equilibrated_time[list(indices)])
top_block_std = stats.binned_statistic(equilibrated_time,equilibrated_top,statistic='std',bins=equilibrated_time[list(indices)])

In [ ]:
steps = 20

In [ ]:
plt.plot(top_block_average.bin_edges[1::steps],top_block_average.statistic[::steps], label='Top monolayer')
plt.fill_between(top_block_average.bin_edges[1::steps],top_block_average.statistic[::steps] - top_block_std.statistic[::steps], top_block_average.statistic[::steps] + top_block_std.statistic[::steps], alpha = 0.5)
plt.plot(bottom_block_average.bin_edges[1::steps],bottom_block_average.statistic[::steps], label='Bottom monolayer')
plt.fill_between(bottom_block_average.bin_edges[1::steps],bottom_block_average.statistic[::steps] - bottom_block_std.statistic[::steps], bottom_block_average.statistic[::steps] + bottom_block_std.statistic[::steps], alpha = 0.5)

plt.ylim(0.4,0.65)
plt.legend(loc='upper right')

In [ ]:
import csv 

with open('equilibrated_data.csv', 'w', newline='') as f:
    writer = csv.writer(f, delimiter='\t')
    writer.writerow(["Time (ps)", "Top", "Top std", "Bot", "Bot std"])
    writer.writerows(zip(top_block_average.bin_edges[1::steps],top_block_average.statistic[::steps],top_block_std.statistic[::steps],bottom_block_average.statistic[::steps],bottom_block_std.statistic[::steps]))

- It is also important to notice that, not every run will generate an exact trajectory, and hence nematic order graph, because there is a random seed for each gromacs run. This random seed information can be extracted from gromacs `*.tpr` file. 
- However, this should not affect the Reproducibility principle in TRUE, because TRUE concerns with the macroscale reproducibility.

### Expanding

- Above is a complete shearing simulation for a single system (a dual-monolayer with a 17 carbon Alkylsilane backbone cap with a methyl terminal group). The ouput result can be used to calculate wide range of tribology properties such as coefficient of friction, force of adhesion, or nematic order (as shown above). However, to study the trends of the system tribological properties induced by the terminal groups, backbone chemistries, or backbone chainlength, etc., we will need to have more than just a few simulations. Hence, design a project with TRUE in mind, will be extremely beneficial in the long run, when we desire to study more variables, extending beyond the original project.
- The following part of this example will try to show how, by designing our code following Object Oriented Programming (OOP) principles and using a few open-source libraries, we can build a TRUE simulation project, emphasizing at its extensibility and scalability. 

In [ ]:
%cd ../example_workspace/

- We start wil initialize a signac workspace by running `init.py` in the `wokrflow/example_workspace/src`.
- In the `init.py` script, we can specify what type of variables we are varying, such as chain length in this example, and create separate directory for each unique system. 
- The script will then create a `signac workspace` comprises of directories, each contains `statepoint` that descibe parameters of corresponding system. 

In [1]:
!python src/init.py --seed 27 --num-replicas 3

python: can't open file 'src/init.py': [Errno 2] No such file or directory


- The `signac workspace` can be then accessed by `signac-flow` to perform different operations as in `workflow/example_workspace/src/project.py`

In [ ]:
!python src/project.py run -o initialize_system

- This example stops at initializing the system, but other steps of the simulations of the workflow can be implemented analogously. 